# Overview

This notebook is to experiment with GitHub API and check whether we can do all we want with Github Issues

Comments:
- When starting to experiment with the API I searched for Python clients and saw that [PyGithub](https://github.com/PyGithub/PyGithub) and [ghapi](https://github.com/AnswerDotAI/ghapi) are used (though not by many people).
- Since our use case is very specific to handle issues, I decided to give it a try developing things from scratch

Step-by-step
1. Create a fine-grained token in Github. Follow these [instructions](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens#creating-a-fine-grained-personal-access-token)
2. Then add permissions to `read and write` issues

# Setup

In [17]:
import requests
import os
import time
import json

In [18]:
os.getcwd()

'c:\\WBG\\wb560793\\TidySDMXProject\\tidysdmx-wt\\github\\notebooks\\github_api'

# Understanding the API

## Load token stored locally with `load_token`

In [19]:
def load_token(file_path, key="WB_DECIS_TOKEN"):
    try:
        with open(file_path, "r") as f:
            for line in f:
                if "=" in line:
                    temp_key, value = line.strip().split("=", 1)
                    if temp_key == key:
                        return value.strip()
        raise ValueError("The token key you provided was not found")
    except FileNotFoundError as e:
        raise FileNotFoundError(f"File not found... {e}")

## Create global variables

In [28]:
# token = load_token("../../gihub.token")
# owner = "WB-DECIS"
# repo = "testing_issues"
token = load_token("../../gihub.token", key="DATA360_DATABRICKS")
owner = "worldbank"
repo = "data360-pipelines-databricks"

In [29]:
# Set up headers with the token
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.github+json",
}

In [30]:
# GitHub API endpoint (example: get authenticated user info)
url = f"https://api.github.com/repos/{owner}/{repo}/issues"

## List issues in a repo

In [31]:
# params
params = {
	"state": "all",  # Options: open, closed, all
	"per_page": 100,  # Number of results per page (max 100)
	"page": 1,       # Page number to retrieve
}

In [32]:
# Make the request
response = requests.get(url, headers=headers, params=params)

# Check the response
if response.status_code == 200:
    print("Authenticated successfully!")
    print(response.json())
else:
    print(f"Failed to authenticate: {response.status_code}")
    print(response.text)


Authenticated successfully!
[{'url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/159', 'repository_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks', 'labels_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/159/labels{/name}', 'comments_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/159/comments', 'events_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/159/events', 'html_url': 'https://github.com/worldbank/data360-pipelines-databricks/issues/159', 'id': 3549737345, 'node_id': 'I_kwDOPFvVMs7TlLGB', 'number': 159, 'title': 'tidsdmx - Type handling in the transform_source_to_target function', 'user': {'login': 'EchoLiuuuuuu', 'id': 63255303, 'node_id': 'MDQ6VXNlcjYzMjU1MzAz', 'avatar_url': 'https://avatars.githubusercontent.com/u/63255303?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/EchoLiuuuuuu', 'html_url': 'http

In [33]:
# Export dict as json
import json

with open("output.json", "w") as f:
    json.dump(response.json(), f)

In [34]:
len(response.json())

100

## Get list of datasets with issues in the repo

In [ ]:
# GitHub API endpoint (example: get authenticated user info)
url = f"https://api.github.com/repos/{owner}/{repo}/issues"

In [38]:
# params
params = {
	"state": "all",  # Options: open, closed, all
	"per_page": 100,  # Number of results per page (max 100)
	"page": 1,       # Page number to retrieve
}

In [39]:
# Iterate through pages
all_issues = []
while True:
	response = requests.get(url, headers=headers, params=params)
	if response.status_code != 200:
		print(f"Failed to retrieve issues: {response.status_code}")
		break

	issues = response.json()
	if not issues:
		break

	all_issues.extend(issues)
	params["page"] += 1
	time.sleep(1)  # To avoid hitting rate limits

In [40]:
print(len(all_issues))
all_issues

159


[{'url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/159',
  'repository_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks',
  'labels_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/159/labels{/name}',
  'comments_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/159/comments',
  'events_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/159/events',
  'html_url': 'https://github.com/worldbank/data360-pipelines-databricks/issues/159',
  'id': 3549737345,
  'node_id': 'I_kwDOPFvVMs7TlLGB',
  'number': 159,
  'title': 'tidsdmx - Type handling in the transform_source_to_target function',
  'user': {'login': 'EchoLiuuuuuu',
   'id': 63255303,
   'node_id': 'MDQ6VXNlcjYzMjU1MzAz',
   'avatar_url': 'https://avatars.githubusercontent.com/u/63255303?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/EchoLiuuuuuu',
   'html_u

In [44]:
# Filter only issues with label "dataset"
dataset_issues = [issue for issue in all_issues if any(label['name'] == 'Dataset' for label in issue.get('labels', []))]
print(len(dataset_issues))
dataset_issues

2


[{'url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/129',
  'repository_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks',
  'labels_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/129/labels{/name}',
  'comments_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/129/comments',
  'events_url': 'https://api.github.com/repos/worldbank/data360-pipelines-databricks/issues/129/events',
  'html_url': 'https://github.com/worldbank/data360-pipelines-databricks/issues/129',
  'id': 3545758873,
  'node_id': 'I_kwDOPFvVMs7TV_yZ',
  'number': 129,
  'title': '[IFC_GBMP] - Green Buildings Market Penetration',
  'user': {'login': 'danielgils',
   'id': 11430123,
   'node_id': 'MDQ6VXNlcjExNDMwMTIz',
   'avatar_url': 'https://avatars.githubusercontent.com/u/11430123?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/danielgils',
   'html_url': 'https://github.co

In [ ]:
# Extract unique dataset ids
dataset_ids = set()
for issue in dataset_issues:
	# Extract title
	title = issue.get('title', '')
	# Extract dataset id from title
	dataset_id = title.split(']')[0].strip("[")
	# Add to set
	dataset_ids.add(dataset_id)
dataset_ids

{'IFC_GBMP', 'WB_ID4D'}

In [41]:

with open("all_issues.json", "w") as f:
    json.dump(all_issues, f)

## Get an issue by number

In [122]:
token = load_token("../../gihub.token")
owner = "WB-DECIS"
repo = "testing_issues"

In [123]:
# Set up headers with the token
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.github+json",
}

In [102]:
# issue_number = 45
# issue_number = 59
issue_number = 88

In [103]:
# GitHub API endpoint (example: get authenticated user info)
url = f"https://api.github.com//repos/{owner}/{repo}/issues/{issue_number}"
url

'https://api.github.com//repos/WB-DECIS/testing_issues/issues/88'

In [104]:
# Make the request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    print("Authenticated successfully!")
    print(response.json())
else:
    print(f"Failed to authenticate: {response.status_code}")
    print(response.text)


Authenticated successfully!
{'url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/88', 'repository_url': 'https://api.github.com/repos/WB-DECIS/testing_issues', 'labels_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/88/labels{/name}', 'comments_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/88/comments', 'events_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/88/events', 'html_url': 'https://github.com/WB-DECIS/testing_issues/issues/88', 'id': 3478871088, 'node_id': 'I_kwDOMdlsPs7PW1ww', 'number': 88, 'title': '[WB_TEST1] - Collection module', 'user': {'login': 'danielgils', 'id': 11430123, 'node_id': 'MDQ6VXNlcjExNDMwMTIz', 'avatar_url': 'https://avatars.githubusercontent.com/u/11430123?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/danielgils', 'html_url': 'https://github.com/danielgils', 'followers_url': 'https://api.github.com/users/danielgils/followers', 'following_url': 'https://api.github.co

In [105]:
len(response.json())

32

In [106]:
# Export dict as json
import json

with open("output.json", "w") as f:
    json.dump(response.json(), f)

## Search issues by string pattern

In [124]:
token = load_token("../../gihub.token")
owner = "WB-DECIS"
repo = "testing_issues"

In [125]:
# Set up headers with the token
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.github+json",
}

In [126]:
search_term = "WB_TEST_2"  # The pattern you're searching for

In [127]:
# GitHub Search API endpoint
url = "https://api.github.com/search/issues"


In [128]:

# Query parameters
params = {
    "q": f"{search_term} in:title repo:{owner}/{repo} is:issue",
}


In [129]:

# Make the request
response = requests.get(url, headers=headers, params=params)

# Check response
if response.status_code == 200:
    issues = response.json()["items"]
    for issue in issues:
        print(f"- #{issue['number']}: {issue['title']} ({issue['html_url']})")
else:
    print(f"Error: {response.status_code}")
    print(response.text)


- #114: [WB_TEST_2] - Harvesting (https://github.com/WB-DECIS/testing_issues/issues/114)
- #112: [WB_TEST_2] - Publication (https://github.com/WB-DECIS/testing_issues/issues/112)
- #113: [WB_TEST_2] - Maintainance (https://github.com/WB-DECIS/testing_issues/issues/113)
- #128: [WB_TEST_2] - Metadata review (https://github.com/WB-DECIS/testing_issues/issues/128)
- #115: [WB_TEST_2] - Data modelling (https://github.com/WB-DECIS/testing_issues/issues/115)
- #108: [WB_TEST_2] - Pipeline integration (https://github.com/WB-DECIS/testing_issues/issues/108)
- #107: [WB_TEST_2] - Dissemination module (https://github.com/WB-DECIS/testing_issues/issues/107)
- #106: [WB_TEST_2] - Processing module (https://github.com/WB-DECIS/testing_issues/issues/106)
- #105: [WB_TEST_2] - Structural metadata (https://github.com/WB-DECIS/testing_issues/issues/105)
- #104: [WB_TEST_2] - Data curation (https://github.com/WB-DECIS/testing_issues/issues/104)
- #103: [WB_TEST_2] - Collection module (https://github.com

In [131]:
response.json()

{'total_count': 30,
 'incomplete_results': False,
 'items': [{'url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/114',
   'repository_url': 'https://api.github.com/repos/WB-DECIS/testing_issues',
   'labels_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/114/labels{/name}',
   'comments_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/114/comments',
   'events_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/114/events',
   'html_url': 'https://github.com/WB-DECIS/testing_issues/issues/114',
   'id': 3481878051,
   'node_id': 'I_kwDOMdlsPs7PiT4j',
   'number': 114,
   'title': '[WB_TEST_2] - Harvesting',
   'user': {'login': 'danielgils',
    'id': 11430123,
    'node_id': 'MDQ6VXNlcjExNDMwMTIz',
    'avatar_url': 'https://avatars.githubusercontent.com/u/11430123?v=4',
    'gravatar_id': '',
    'url': 'https://api.github.com/users/danielgils',
    'html_url': 'https://github.com/danielgils',
    'followers_url': '

In [132]:
# Export dict as json
import json

with open("output.json", "w") as f:
    json.dump(response.json(), f)

In [136]:
issues_found = []
for issue in response.json().get("items", []):
    issues_found.append({
		"id": issue.get("id"),
		"node_id": issue.get("node_id"),
		"number": issue.get("number"),
		"title": issue.get("title"),
        "labels": [label.get("name") for label in issue.get("labels", [])],
		"url": issue.get("html_url"),
		"state": issue.get("state"),
	})

In [137]:
issues_found

[{'id': 3481878051,
  'node_id': 'I_kwDOMdlsPs7PiT4j',
  'number': 114,
  'title': '[WB_TEST_2] - Harvesting',
  'labels': ['Task', 'Collection'],
  'url': 'https://github.com/WB-DECIS/testing_issues/issues/114',
  'state': 'open'},
 {'id': 3481877995,
  'node_id': 'I_kwDOMdlsPs7PiT3r',
  'number': 112,
  'title': '[WB_TEST_2] - Publication',
  'labels': ['Epic', 'Publishing'],
  'url': 'https://github.com/WB-DECIS/testing_issues/issues/112',
  'state': 'open'},
 {'id': 3481878019,
  'node_id': 'I_kwDOMdlsPs7PiT4D',
  'number': 113,
  'title': '[WB_TEST_2] - Maintainance',
  'labels': ['Epic', 'Maintenance'],
  'url': 'https://github.com/WB-DECIS/testing_issues/issues/113',
  'state': 'open'},
 {'id': 3481878746,
  'node_id': 'I_kwDOMdlsPs7PiUDa',
  'number': 128,
  'title': '[WB_TEST_2] - Metadata review',
  'labels': ['Task', 'Metadata review'],
  'url': 'https://github.com/WB-DECIS/testing_issues/issues/128',
  'state': 'open'},
 {'id': 3481878077,
  'node_id': 'I_kwDOMdlsPs7PiT49',

## Create an issue in a repo

In [117]:
# GitHub API endpoint (example: get authenticated user info)
url = f"https://api.github.com//repos/{owner}/{repo}/issues"

In [ ]:
dataset_id = "WB_TEST_2"
dataset_name = "WB TEST 2 using API"

In [ ]:
# # # # # params
# # # create_params = {
# # # 	"title": f"[{dataset_id}] - WB TEST 1 using API",
# # # 	# "title": f"[TESTEST] - WB TEST 1 using API",
# # # 	"labels": ["Dataset"],
# # # }

# create_params = {
# 	# "title": f"[{dataset_id}] - WB TEST 1 using API",
# 	"title": f"[Manual test2] - Harvesting",
# 	"labels": ["Collection", "Task"],
#     "body": "- [ ] Implement/Document collection function (API, File)\n- [ ] Store raw data and data dictionary (list of indicators) in defined location (possibly DDH)\n- [ ] Share list of indicator details (data and metadata available) with Curator",
#     "type": "Task",
# }

In [ ]:
# # Make the request
# response = requests.post(url, headers=headers, json=create_params)

# # Check the response
# if response.status_code in [200, 201]:
# 	print("Issue created successfully!")
# 	print(response.json())
# 	issue_number = response.json().get("number")
# 	print(f"Issue number: {issue_number}")
# else:
# 	print(f"Could not create issue: {response.status_code}")
# 	print(response.text)


Issue created successfully!
{'url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/133', 'repository_url': 'https://api.github.com/repos/WB-DECIS/testing_issues', 'labels_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/133/labels{/name}', 'comments_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/133/comments', 'events_url': 'https://api.github.com/repos/WB-DECIS/testing_issues/issues/133/events', 'html_url': 'https://github.com/WB-DECIS/testing_issues/issues/133', 'id': 3481933429, 'node_id': 'I_kwDOMdlsPs7PihZ1', 'number': 133, 'title': '[Manual test2] - Harvesting', 'user': {'login': 'danielgils', 'id': 11430123, 'node_id': 'MDQ6VXNlcjExNDMwMTIz', 'avatar_url': 'https://avatars.githubusercontent.com/u/11430123?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/danielgils', 'html_url': 'https://github.com/danielgils', 'followers_url': 'https://api.github.com/users/danielgils/followers', 'following_url': 'https://api.github

In [ ]:
# # Export dict as json
# import json

# with open("output.json", "w") as f:
#     json.dump(response.json(), f)

In [ ]:
# issue_number = response.json().get("number")
# issue_number

87

In [75]:
issues_list = [
	{
		"title": f"[{dataset_id}] - {dataset_name}",
		"labels": ["Dataset"]
	},
	{
		"title": f"[{dataset_id}] - Collection module",
		"labels": ["Collection", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Data curation",
		"labels": ["Modeling", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Structural metadata",
		"labels": ["FMR", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Processing module",
		"labels": ["Processing", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Dissemination module",
		"labels": ["Dissemination", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Pipeline integration",
		"labels": ["Pip. integration", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Pipeline to staging",
		"labels": ["Pip. to staging", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Referential metadata review",
		"labels": ["Metadata review", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Data and metadata approval",
		"labels": ["Meta-data approval", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Publication",
		"labels": ["Publishing", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Maintainance",
		"labels": ["Maintenance", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Harvesting",
		"labels": ["Collection", "Task"],
		"body": "- [ ] Implement/Document collection function (API, File)\n- [ ] Store raw data and data dictionary (list of indicators) in defined location (possibly DDH)\n- [ ] Share list of indicator details (data and metadata available) with Curator",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Data modelling",
		"labels": ["Modeling", "Task"],
		"body": "- [ ] Evaluation of the data to identify dimensionality, attributes and collapsing options.\n- [ ] Re-modeling design to comply with the established standards (DSDs, codelists, etc.)\n- [ ] Document finalized dimensions and items code(s) and name(s) in the template (Matrix generator or other - pending)",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Process artifacts in FMR",
		"labels": ["FMR", "Task"],
		"body": "- [ ] Process (update/upload) artifacts in FMR",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Implement/Document specifications for transformations (collapsing and new columns)",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Implement/Document aggregation processing",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Store processed data (Azure datalake or Unity Catalog)",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Hard check validation",
		"labels": ["Processing", "Task"],
		"body": "- [ ] Implement hard check validation\n- [ ] Create report about (hard check) validation\n- [ ] Implement automatic (code) tests and log results",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Implement process validation reports (optional)",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Test processing system",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Apply instruments to ensure ingestion of finalized data into dissemination environment",
		"labels": ["Dissemination", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Prepopulate referencial metadata from mapping file to ME",
		"labels": ["Dissemination", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Test dissemination system",
		"labels": ["Dissemination", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Create complete pipeline",
		"labels": ["Pip. integration", "Task"],
        "body": "- [ ] Create pipeline with all components\n- [ ] Testing results and evaluate potential improvements when needed\n- [ ] Pipeline request to move to production environment",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Move to staging",
		"labels": ["Pip. to staging", "Task"],
        "body": "- [ ] Pipeline review and approval to staging\n- [ ] Pipeline move to staging\n- [ ] Initial pipeline execution to push data into STG",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Metadata review",
		"labels": ["Metadata review", "Task"],
        "body": "- [ ] Ensure referential metadata is ingested\n- [ ] Request to FP or DR to gather their inputs on missing fields or improvement in ME projects\n- [ ] Apply or request to FP or DR to assign tags to indicators ",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Data and metadata approval",
		"labels": ["Meta-data approval", "Task"],
        "body": "- [ ] Request for approval from DR or FP\n- [ ] Document response\n- [ ] Publish data and metadata in staging site\n- [ ] Data AND metadata approved?",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Move to Production",
		"labels": ["Publishing", "Task"],
        "body": "- [ ] Move code to production\n- [ ] Move data to prod: Push data from STG to Dissemination environment\n- [ ] Schedule pipeline",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Add pipeline to monitoring tool",
		"labels": ["Maintenance", "Task"],
		"type": "Task",
	},
]

In [76]:
len(issues_list)

30

In [92]:
# Initialize list to store created issues
created_issues = []

# Adding issues to repo
for issue in issues_list:
	print(f'{issue["title"]} - {issue["labels"]}')
	# Create issue
	response = requests.post(url, headers=headers, json=issue)

	# Check the response
	if response.status_code in [200, 201]:
		print("Issue created successfully!")
		# print(response.json())
		issue["id"] = response.json().get("id")
		issue["node_id"] = response.json().get("node_id")
		issue["number"] = response.json().get("number")
		# print(f"Issue number: {issue["issue_number"]}")
		time.sleep(0.1)
		created_issues.append(issue)
	else:
		print(f"Could not create issue: {response.status_code}")
		print(response.text)

# Export created issues as json
with open("created_issues.json", "w") as f:
	json.dump(created_issues, f)


[WB_TEST_2] - WB TEST 2 using API - ['Dataset']
Issue created successfully!
[WB_TEST_2] - Collection module - ['Collection', 'Epic']
Issue created successfully!
[WB_TEST_2] - Data curation - ['Modeling', 'Epic']
Issue created successfully!
[WB_TEST_2] - Structural metadata - ['FMR', 'Epic']
Issue created successfully!
[WB_TEST_2] - Processing module - ['Processing', 'Epic']
Issue created successfully!
[WB_TEST_2] - Dissemination module - ['Dissemination', 'Epic']
Issue created successfully!
[WB_TEST_2] - Pipeline integration - ['Pip. integration', 'Epic']
Issue created successfully!
[WB_TEST_2] - Pipeline to staging - ['Pip. to staging', 'Epic']
Issue created successfully!
[WB_TEST_2] - Referential metadata review - ['Metadata review', 'Epic']
Issue created successfully!
[WB_TEST_2] - Data and metadata approval - ['Meta-data approval', 'Epic']
Issue created successfully!
[WB_TEST_2] - Publication - ['Publishing', 'Epic']
Issue created successfully!
[WB_TEST_2] - Maintainance - ['Mainte

## Add subissues

In [107]:
# Set up headers with the token
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.github+json",
}

In [ ]:
dataset_issue_number = 87
epic_issue_id = 3478871088

In [109]:
# url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/sub_issues"
url = f"https://api.github.com/repos/{owner}/{repo}/issues/{dataset_issue_number}/sub_issues"
url

'https://api.github.com/repos/WB-DECIS/testing_issues/issues/87/sub_issues'

In [ ]:
# params
create_params = {
	"sub_issue_id": epic_issue_id,
    "replace_parent": True,
}

In [112]:
# Make the request
response = requests.post(url, headers=headers, json=create_params)

# Check the response
if response.status_code in [200, 201]:
    print("Subissue added successfully!")
    print(response.json())
else:
    print(f"Could not add issue: {response.status_code}")
    print(response.text)


Could not add issue: 422
{"message":"An error occurred while adding the sub-issue to the parent issue. Issue may not contain duplicate sub-issues","documentation_url":"https://docs.github.com/rest/issues/sub-issues#add-sub-issue","status":"422"}


In [138]:
# For now, just assigning issues I found by title, but the idea is to use
# created issues object I created when creating the issues
created_issues = issues_found

In [142]:
# Create subsets by tasks, epics and datasets labels
datasets = [issue for issue in created_issues if "Dataset" in issue["labels"]]
epics = [issue for issue in created_issues if "Epic" in issue["labels"]]
tasks = [issue for issue in created_issues if "Task" in issue["labels"]]

In [145]:
dataset_issue_number = datasets[0]["number"]
dataset_issue_number

102

In [ ]:
url = f"https://api.github.com/repos/{owner}/{repo}/issues/{dataset_issue_number}/sub_issues"

In [147]:
# Assign epics as sub-issues to dataset
for epic in epics:
	print(f'Adding epic {epic["title"]} to dataset issue #{dataset_issue_number}')
	epic_issue_id = epic["id"]
	# params
	create_params = {
		"sub_issue_id": epic_issue_id,
		"replace_parent": False,
	}

	# Make the request
	response = requests.post(url, headers=headers, json=create_params)

	# Check the response
	if response.status_code in [200, 201]:
		print("Subissue added successfully!")
		# print(response.json())
	else:
		print(f"Could not add issue: {response.status_code}")
		print(response.text)
	time.sleep(0.1)

Adding epic [WB_TEST_2] - Publication to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Maintainance to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Pipeline integration to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Dissemination module to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Processing module to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Structural metadata to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Data curation to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Collection module to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Pipeline to staging to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Referential metadata review to dataset issue #102
Subissue added successfully!
Adding epic [WB_TEST_2] - Data and metadata a

In [150]:
[label for label in epic["labels"] if label != "Epic"][0]

'Meta-data approval'

In [151]:
# Assign tasks as sub-issues to epics
for epic in epics:
	# Extract issue number and primary label
	issue_number = epic["number"]
	label = [label for label in epic["labels"] if label != "Epic"][0]
	
	# Filter tasks that match the epic's primary label
	tasks_for_epic = [task for task in tasks if label in task["labels"]]

	# Update url for each epic
	url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/sub_issues"
	
	# Loop through tasks and assign to epic
	for task in tasks_for_epic:
		print(f'Adding epic {task["title"]} to epic issue #{issue_number} - label: {label}')
		task_issue_id = task["id"]
		# params
		create_params = {
			"sub_issue_id": task_issue_id,
			"replace_parent": False,
		}

		# Make the request
		response = requests.post(url, headers=headers, json=create_params)

		# Check the response
		if response.status_code in [200, 201]:
			print("Subissue added successfully!")
			# print(response.json())
		else:
			print(f"Could not add issue: {response.status_code}")
			print(response.text)
		time.sleep(0.1)

Adding epic [WB_TEST_2] - Move to Production to epic issue #112 - label: Publishing
Subissue added successfully!
Adding epic [WB_TEST_2] - Add pipeline to monitoring tool to epic issue #113 - label: Maintenance
Subissue added successfully!
Adding epic [WB_TEST_2] - Create complete pipeline to epic issue #108 - label: Pip. integration
Subissue added successfully!
Adding epic [WB_TEST_2] - Test dissemination system to epic issue #107 - label: Dissemination
Subissue added successfully!
Adding epic [WB_TEST_2] - Prepopulate referencial metadata from mapping file to ME to epic issue #107 - label: Dissemination
Subissue added successfully!
Adding epic [WB_TEST_2] - Apply instruments to ensure ingestion of finalized data into dissemination environment to epic issue #107 - label: Dissemination
Subissue added successfully!
Adding epic [WB_TEST_2] - Test processing system to epic issue #106 - label: Processing
Subissue added successfully!
Adding epic [WB_TEST_2] - Hard check validation to epic i

# Putting everything together

## Create issues for a dataset

In [ ]:
# token = load_token("../../gihub.token")
# owner = "WB-DECIS"
# repo = "testing_issues"

In [11]:
token = load_token("../../gihub.token", key="DATA360_DATABRICKS")
owner = "worldbank"
repo = "data360-pipelines-databricks"

In [5]:
# Set up headers with the token
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.github+json",
}

In [6]:
# GitHub API endpoint (example: get authenticated user info)
url = f"https://api.github.com//repos/{owner}/{repo}/issues"

In [7]:
# dataset_id = "WB_TEST_4"
# dataset_name = "WB TEST 4 using API"
# dataset_id = "WB_ID4D"
# dataset_name = "ID4D Global Dataset"
dataset_id = "IFC_GBMP"
dataset_name = "Green Buildings Market Penetration"

In [8]:
issues_list = [
	{
		"title": f"[{dataset_id}] - {dataset_name}",
		"labels": ["Dataset"]
	},
	{
		"title": f"[{dataset_id}] - Collection module",
		"labels": ["Collection", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Data curation",
		"labels": ["Modeling", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Metadata elements creation",
		"labels": ["Metadata elements", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Processing module",
		"labels": ["Processing", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Pipeline integration",
		"labels": ["Pip. integration", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Pipeline to prod",
		"labels": ["Pip. to prod", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Referential metadata review",
		"labels": ["Metadata review", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Data and metadata approval",
		"labels": ["Meta-data approval", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Publication",
		"labels": ["Publishing", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Maintainance",
		"labels": ["Maintenance", "Epic"],
	},
	{
		"title": f"[{dataset_id}] - Harvesting",
		"labels": ["Collection", "Task"],
		"body": "- [ ] Implement/Document collection function (API, File)\n- [ ] Store raw data and data dictionary (list of indicators) in defined location (possibly DDH)\n- [ ] Share list of indicator details (data and metadata available) with Curator",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Data modelling",
		"labels": ["Modeling", "Task"],
		"body": "- [ ] Evaluation of the data to identify dimensionality, attributes and collapsing options.\n- [ ] Re-modeling design to comply with the established standards (DSDs, codelists, etc.)\n- [ ] Document finalized dimensions and items code(s) and name(s) in the template.",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Structural metadata: Process artifacts in FMR",
		"labels": ["Metadata elements", "Task"],
		"body": "- [ ] Process (update/upload) artifacts in FMR",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Referencial metadata: Create projects in metadata editor",
		"labels": ["Metadata elements", "Task"],
		"body": "- [ ] Create indicator and dataset metadata projects in Metadata Editor",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Send notification to data owners",
		"labels": ["Metadata elements", "Task"],
		"body": "- [ ] Send notification to data owners that all projects have been created in the Metadata Editor",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Implement specifications for transformations (collapsing and new columns)",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Implement/Document aggregation processing",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Store processed data (Silver)",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Structural validation",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Store processed data (Gold)",
		"labels": ["Processing", "Task"],
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Content validation",
		"labels": ["Processing", "Task"],
		"body": "- [ ] Implement hard check validation\n- [ ] Create report about (hard check) validation\n- [ ] Implement automatic (code) tests and log results",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Test processing system",
		"labels": ["Processing", "Task"],
        "body": "- [ ] Test processing system.\n- [ ] Create PR to main.",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - PR Review: Testing custom tasks until validation",
		"labels": ["Pip. integration", "Task"],
        "body": "- [ ] PR review: Testing custom tasks until validation (locally).",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Enable publish pipeline",
		"labels": ["Pip. integration", "Task"],
        "body": "- [ ] Apply instruments to ensure ingestion of finalized data into dissemination environment.\n- [ ] Prepopulate referencial metadata from mapping file to ME.\n- [ ] Create Indicators CSV files.\n- [ ] Create Download CSV files.\n- [ ] Export JSON and PDF metadata files.\n- [ ] Verify exported files.\n- [ ] Trigger publishing.",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Move pipeline to production",
		"labels": ["Pip. to prod", "Task"],
        "body": "- [ ] Push pipeline to prod in Databricks.\n- [ ] Initial pipeline execution to push data into PROD.",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Metadata review",
		"labels": ["Metadata review", "Task"],
        "body": "- [ ] Ensure referential metadata is ingested\n- [ ] Request to FP or DR to gather their inputs on missing fields or improvement in ME projects\n- [ ] Apply or request to FP or DR to assign tags to indicators.",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Data and metadata approval",
		"labels": ["Meta-data approval", "Task"],
        "body": "- [ ] Request for approval from DR or FP\n- [ ] Document response in GitHub\n- [ ] Data AND metadata approved?",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Move to Production",
		"labels": ["Publishing", "Task"],
        "body": "- [ ] Evaluate embargo and apply it.\n- [ ] Trigger change from staging to prod.\n- [ ] Schedule pipeline",
		"type": "Task",
	},
    {
		"title": f"[{dataset_id}] - Add pipeline to monitoring tool",
		"labels": ["Maintenance", "Task"],
		"type": "Task",
	},
]

In [9]:
# Initialize list to store created issues
created_issues = []

# Adding issues to repo
for issue in issues_list:
	print(f'{issue["title"]} - {issue["labels"]}')
	# Create issue
	response = requests.post(url, headers=headers, json=issue)

	# Check the response
	if response.status_code in [200, 201]:
		print("Issue created successfully!")
		# print(response.json())
		issue["id"] = response.json().get("id")
		issue["node_id"] = response.json().get("node_id")
		issue["number"] = response.json().get("number")
		# print(f"Issue number: {issue["issue_number"]}")
		created_issues.append(issue)
	else:
		print(f"Could not create issue: {response.status_code}")
		print(response.text)
	time.sleep(0.1)

# Export created issues as json
with open("created_issues.json", "w") as f:
	json.dump(created_issues, f)


[IFC_GBMP] - Green Buildings Market Penetration - ['Dataset']
Issue created successfully!
[IFC_GBMP] - Collection module - ['Collection', 'Epic']
Issue created successfully!
[IFC_GBMP] - Data curation - ['Modeling', 'Epic']
Issue created successfully!
[IFC_GBMP] - Metadata elements creation - ['Metadata elements', 'Epic']
Issue created successfully!
[IFC_GBMP] - Processing module - ['Processing', 'Epic']
Issue created successfully!
[IFC_GBMP] - Pipeline integration - ['Pip. integration', 'Epic']
Issue created successfully!
[IFC_GBMP] - Pipeline to prod - ['Pip. to prod', 'Epic']
Issue created successfully!
[IFC_GBMP] - Referential metadata review - ['Metadata review', 'Epic']
Issue created successfully!
[IFC_GBMP] - Data and metadata approval - ['Meta-data approval', 'Epic']
Issue created successfully!
[IFC_GBMP] - Publication - ['Publishing', 'Epic']
Issue created successfully!
[IFC_GBMP] - Maintainance - ['Maintenance', 'Epic']
Issue created successfully!
[IFC_GBMP] - Harvesting - ['

## Add subissues

In [ ]:
# token = load_token("../../gihub.token")
# owner = "WB-DECIS"
# repo = "testing_issues"

In [10]:
token = load_token("../../gihub.token", key="DATA360_DATABRICKS")
owner = "worldbank"
repo = "data360-pipelines-databricks"

In [11]:
# Set up headers with the token
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.github+json",
}

In [12]:
# Create subsets by tasks, epics and datasets labels
datasets = [issue for issue in created_issues if "Dataset" in issue["labels"]]
epics = [issue for issue in created_issues if "Epic" in issue["labels"]]
tasks = [issue for issue in created_issues if "Task" in issue["labels"]]

In [13]:
dataset_issue_number = datasets[0]["number"]
dataset_issue_number

129

In [14]:
url = f"https://api.github.com/repos/{owner}/{repo}/issues/{dataset_issue_number}/sub_issues"

In [15]:
# Assign epics as sub-issues to dataset
for epic in epics:
	print(f'Adding epic {epic["title"]} to dataset issue #{dataset_issue_number}')
	epic_issue_id = epic["id"]
	# params
	create_params = {
		"sub_issue_id": epic_issue_id,
		"replace_parent": False,
	}

	# Make the request
	response = requests.post(url, headers=headers, json=create_params)

	# Check the response
	if response.status_code in [200, 201]:
		print("Subissue added successfully!")
		# print(response.json())
	else:
		print(f"Could not add issue: {response.status_code}")
		print(response.text)
	time.sleep(0.1)

Adding epic [IFC_GBMP] - Collection module to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Data curation to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Metadata elements creation to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Processing module to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Pipeline integration to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Pipeline to prod to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Referential metadata review to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Data and metadata approval to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Publication to dataset issue #129
Subissue added successfully!
Adding epic [IFC_GBMP] - Maintainance to dataset issue #129
Subissue added successfully!


In [16]:
# Assign tasks as sub-issues to epics
for epic in epics:
	# Extract issue number and primary label
	issue_number = epic["number"]
	label = [label for label in epic["labels"] if label != "Epic"][0]
	
	# Filter tasks that match the epic's primary label
	tasks_for_epic = [task for task in tasks if label in task["labels"]]

	# Update url for each epic
	url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/sub_issues"
	
	# Loop through tasks and assign to epic
	for task in tasks_for_epic:
		print(f'Adding epic {task["title"]} to epic issue #{issue_number} - label: {label}')
		task_issue_id = task["id"]
		# params
		create_params = {
			"sub_issue_id": task_issue_id,
			"replace_parent": False,
		}

		# Make the request
		response = requests.post(url, headers=headers, json=create_params)

		# Check the response
		if response.status_code in [200, 201]:
			print("Subissue added successfully!")
			# print(response.json())
		else:
			print(f"Could not add issue: {response.status_code}")
			print(response.text)
		time.sleep(0.1)

Adding epic [IFC_GBMP] - Harvesting to epic issue #130 - label: Collection
Subissue added successfully!
Adding epic [IFC_GBMP] - Data modelling to epic issue #131 - label: Modeling
Subissue added successfully!
Adding epic [IFC_GBMP] - Structural metadata: Process artifacts in FMR to epic issue #132 - label: Metadata elements
Subissue added successfully!
Adding epic [IFC_GBMP] - Referencial metadata: Create projects in metadata editor to epic issue #132 - label: Metadata elements
Subissue added successfully!
Adding epic [IFC_GBMP] - Send notification to data owners to epic issue #132 - label: Metadata elements
Subissue added successfully!
Adding epic [IFC_GBMP] - Implement specifications for transformations (collapsing and new columns) to epic issue #133 - label: Processing
Subissue added successfully!
Adding epic [IFC_GBMP] - Implement/Document aggregation processing to epic issue #133 - label: Processing
Subissue added successfully!
Adding epic [IFC_GBMP] - Store processed data (Silve

# Developing functions

## extract_list_dataset_ids

In [ ]:
def extract_list_dataset_ids(
		token: str,
		owner: str="worldbank",
		repo: str="data360-pipelines-databricks",
		label: str="Dataset"
	) -> set:
	"""Function to get a set of dataset IDs from GitHub issues in a repo that 
	have the specified label.
	
	Args:
		token (str): Personal access token for GitHub API authentication.
		owner (str): GitHub repository owner.
		repo (str): GitHub repository name.
		label (str): Label to filter issues by (default is "Dataset").
	Returns:
		set: A set of unique dataset IDs extracted from issue titles.
	"""
	# Set up headers with the token
	headers = {
		"Authorization": f"Bearer {token}",
		"Accept": "application/vnd.github+json",
	}

	# GitHub API endpoint (example: get authenticated user info)
	url = f"https://api.github.com/repos/{owner}/{repo}/issues"	

	# params
	params = {
		"state": "all",  # Options: open, closed, all
		"per_page": 100,  # Number of results per page (max 100)
		"page": 1,       # Page number to retrieve
	}

	# Iterate through pages
	print("Retrieving issues from GitHub...")
	all_issues = []
	while True:
		response = requests.get(url, headers=headers, params=params)
		if response.status_code != 200:
			print(f"Failed to retrieve issues: {response.status_code}")
			break

		issues = response.json()
		if not issues:
			break

		all_issues.extend(issues)
		params["page"] += 1
		time.sleep(1)  # To avoid hitting rate limits

	# Filter only issues with label "dataset"
	dataset_issues = [issue for issue in all_issues if 
					any(labels['name'] == label for labels in 
		   			issue.get('labels', []))]
	
	# Extract unique dataset ids
	print("Extracting dataset IDs from issues...")
	dataset_ids = set()
	for issue in dataset_issues:
		# Extract title
		title = issue.get('title', '')
		# Extract dataset id from title
		dataset_id = title.split(']')[0].strip("[")
		# Add to set
		dataset_ids.add(dataset_id)
	return dataset_ids
	

In [59]:
token = load_token("../../gihub.token", key="DATA360_DATABRICKS")
owner = "worldbank"
repo = "data360-pipelines-databricks"

In [60]:
dataset_ids = extract_list_dataset_ids(token=token, owner=owner, repo=repo)
dataset_ids

Retrieving issues from GitHub...
Extracting dataset IDs from issues...


{'IFC_GBMP', 'WB_ID4D'}

## create_issues

In [66]:
def create_issues(
		dataset_id: str,
		dataset_name: str,
		token: str,
		owner: str="worldbank",
		repo: str="data360-pipelines-databricks"
	) -> list:
	"""Function to create GitHub issues for a dataset in a specified repo.

	This function creates issues based on the a predefined structure for dataset.

	Args:
		dataset_id (str): Dataset identifier.
		dataset_name (str): Dataset name.
		token (str): Personal access token for GitHub API authentication.
		owner (str): GitHub repository owner.
		repo (str): GitHub repository name.

	Returns:
		list: A list of created issues with their details.
	"""
	# Set up headers with the token
	headers = {
		"Authorization": f"Bearer {token}",
		"Accept": "application/vnd.github+json",
	}

	# GitHub API endpoint to create issues
	url = f"https://api.github.com/repos/{owner}/{repo}/issues"

	# Define issue list with details
	issues_list = [
		{
			"title": f"[{dataset_id}] - {dataset_name}",
			"labels": ["Dataset"]
		},
		{
			"title": f"[{dataset_id}] - Collection module",
			"labels": ["Collection", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Data curation",
			"labels": ["Modeling", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Metadata elements creation",
			"labels": ["Metadata elements", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Processing module",
			"labels": ["Processing", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Pipeline integration",
			"labels": ["Pip. integration", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Pipeline to prod",
			"labels": ["Pip. to prod", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Referential metadata review",
			"labels": ["Metadata review", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Data and metadata approval",
			"labels": ["Meta-data approval", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Publication",
			"labels": ["Publishing", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Maintainance",
			"labels": ["Maintenance", "Epic"],
		},
		{
			"title": f"[{dataset_id}] - Harvesting",
			"labels": ["Collection", "Task"],
			"body": "- [ ] Implement/Document collection function (API, File)\n- [ ] Store raw data and data dictionary (list of indicators) in defined location (possibly DDH)\n- [ ] Share list of indicator details (data and metadata available) with Curator",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Data modelling",
			"labels": ["Modeling", "Task"],
			"body": "- [ ] Evaluation of the data to identify dimensionality, attributes and collapsing options.\n- [ ] Re-modeling design to comply with the established standards (DSDs, codelists, etc.)\n- [ ] Document finalized dimensions and items code(s) and name(s) in the template.",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Structural metadata: Process artifacts in FMR",
			"labels": ["Metadata elements", "Task"],
			"body": "- [ ] Process (update/upload) artifacts in FMR",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Referencial metadata: Create projects in metadata editor",
			"labels": ["Metadata elements", "Task"],
			"body": "- [ ] Create indicator and dataset metadata projects in Metadata Editor",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Send notification to data owners",
			"labels": ["Metadata elements", "Task"],
			"body": "- [ ] Send notification to data owners that all projects have been created in the Metadata Editor",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Implement specifications for transformations (collapsing and new columns)",
			"labels": ["Processing", "Task"],
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Implement/Document aggregation processing",
			"labels": ["Processing", "Task"],
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Store processed data (Silver)",
			"labels": ["Processing", "Task"],
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Structural validation",
			"labels": ["Processing", "Task"],
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Store processed data (Gold)",
			"labels": ["Processing", "Task"],
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Content validation",
			"labels": ["Processing", "Task"],
			"body": "- [ ] Implement hard check validation\n- [ ] Create report about (hard check) validation\n- [ ] Implement automatic (code) tests and log results",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Test processing system",
			"labels": ["Processing", "Task"],
			"body": "- [ ] Test processing system.\n- [ ] Create PR to main.",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - PR Review: Testing custom tasks until validation",
			"labels": ["Pip. integration", "Task"],
			"body": "- [ ] PR review: Testing custom tasks until validation (locally).",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Enable publish pipeline",
			"labels": ["Pip. integration", "Task"],
			"body": "- [ ] Apply instruments to ensure ingestion of finalized data into dissemination environment.\n- [ ] Prepopulate referencial metadata from mapping file to ME.\n- [ ] Create Indicators CSV files.\n- [ ] Create Download CSV files.\n- [ ] Export JSON and PDF metadata files.\n- [ ] Verify exported files.\n- [ ] Trigger publishing.",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Move pipeline to production",
			"labels": ["Pip. to prod", "Task"],
			"body": "- [ ] Push pipeline to prod in Databricks.\n- [ ] Initial pipeline execution to push data into PROD.",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Metadata review",
			"labels": ["Metadata review", "Task"],
			"body": "- [ ] Ensure referential metadata is ingested\n- [ ] Request to FP or DR to gather their inputs on missing fields or improvement in ME projects\n- [ ] Apply or request to FP or DR to assign tags to indicators.",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Data and metadata approval",
			"labels": ["Meta-data approval", "Task"],
			"body": "- [ ] Request for approval from DR or FP\n- [ ] Document response in GitHub\n- [ ] Data AND metadata approved?",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Move to Production",
			"labels": ["Publishing", "Task"],
			"body": "- [ ] Evaluate embargo and apply it.\n- [ ] Trigger change from staging to prod.\n- [ ] Schedule pipeline",
			"type": "Task",
		},
		{
			"title": f"[{dataset_id}] - Add pipeline to monitoring tool",
			"labels": ["Maintenance", "Task"],
			"type": "Task",
		},
	]

	# Initialize list to store created issues
	created_issues = []

	# Adding issues to repo
	for issue in issues_list:
		print(f'{issue["title"]} - {issue["labels"]}')
		# Create issue
		response = requests.post(url, headers=headers, json=issue)

		# Check the response
		if response.status_code in [200, 201]:
			print("Issue created successfully!")
			# print(response.json())
			issue["id"] = response.json().get("id")
			issue["node_id"] = response.json().get("node_id")
			issue["number"] = response.json().get("number")
			# print(f"Issue number: {issue["issue_number"]}")
			created_issues.append(issue)
		else:
			print(f"Could not create issue: {response.status_code}")
			print(response.text)
		time.sleep(0.1)
	# End for loop

	return created_issues


## add_subissues

In [67]:
def add_subissues(
		created_issues: list, 
		token: str,
		owner: str="worldbank",
		repo: str="data360-pipelines-databricks"
	):
	"""Function to add sub-issues to created issues in a GitHub repo.

	This function add epics as sub-issues to dataset issue and tasks as sub-issues to epics.
	
	Args:
		created_issues (list): List of created issues with their details.
		token (str): Personal access token for GitHub API authentication.
		owner (str): GitHub repository owner.
		repo (str): GitHub repository name.
	Returns:
		Bool: True if sub-issues were added successfully, False otherwise.
	"""
	# Set up headers with the token
	headers = {
		"Authorization": f"Bearer {token}",
		"Accept": "application/vnd.github+json",
	}

	# Create subsets by tasks, epics and datasets labels
	datasets = [issue for issue in created_issues if "Dataset" in issue["labels"]]
	epics = [issue for issue in created_issues if "Epic" in issue["labels"]]
	tasks = [issue for issue in created_issues if "Task" in issue["labels"]]

	# Extract dataset issue number
	dataset_issue_number = datasets[0]["number"]

	# GitHub API endpoint to add subissues to specified issue
	url = f"https://api.github.com/repos/{owner}/{repo}/issues/{dataset_issue_number}/sub_issues"

	# Assign epics as sub-issues to dataset
	for epic in epics:
		print(f'Adding epic {epic["title"]} to dataset issue #{dataset_issue_number}')
		epic_issue_id = epic["id"]
		# params
		create_params = {
			"sub_issue_id": epic_issue_id,
			"replace_parent": False,
		}

		# Make the request
		response = requests.post(url, headers=headers, json=create_params)

		# Check the response
		if response.status_code in [200, 201]:
			print("Subissue added successfully!")
			# print(response.json())
		else:
			print(f"Could not add issue: {response.status_code}")
			print(response.text)
		time.sleep(0.1)
	# End for loop through epics

	# Assign tasks as sub-issues to epics
	for epic in epics:
		# Extract issue number and primary label
		issue_number = epic["number"]
		label = [label for label in epic["labels"] if label != "Epic"][0]
		
		# Filter tasks that match the epic's primary label
		tasks_for_epic = [task for task in tasks if label in task["labels"]]

		# Update url for each epic
		url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/sub_issues"
		
		# Loop through tasks and assign to epic
		for task in tasks_for_epic:
			print(f'Adding epic {task["title"]} to epic issue #{issue_number} - label: {label}')
			task_issue_id = task["id"]
			# params
			create_params = {
				"sub_issue_id": task_issue_id,
				"replace_parent": False,
			}

			# Make the request
			response = requests.post(url, headers=headers, json=create_params)

			# Check the response
			if response.status_code in [200, 201]:
				print("Subissue added successfully!")
				# print(response.json())
			else:
				print(f"Could not add issue: {response.status_code}")
				print(response.text)
			time.sleep(0.1)
		# End for loop through tasks
	# End for loop through epics

	return True

## create_issues_for_dataset

In [61]:
def create_issues_for_dataset(
		dataset_id: str,
		dataset_name: str,
		token: str,
		owner: str="worldbank",
		repo: str="data360-pipelines-databricks"
	) -> None | bool:
	"""
	Args:
	Returns:
	"""
	print("Extracting the list of created datasets in the repo...")
	existing_dataset_ids = extract_list_dataset_ids(
		token=token,
		owner=owner,
		repo=repo,
		label="Dataset"
	)
	
	# Check if dataset_id already exists
	if dataset_id not in existing_dataset_ids:
		print(f"Creating issue for dataset ID {dataset_id}...")
		# Create issues
		issues = create_issues(
			dataset_id=dataset_id,
			dataset_name=dataset_name,
			token=token,
			owner=owner,
			repo=repo
		)

		# Add subissues
		added = add_subissues(
			created_issues=issues,
			token=token,
			owner=owner,
			repo=repo
		)

		return added
	else:
		print(f"Dataset ID {dataset_id} already exists. Skipping issue creation.")
		return None

In [62]:
dataset_id = "WB_TEST_4"
dataset_name = "WB TEST 4 using API"
token = load_token("../../gihub.token")
owner = "WB-DECIS"
repo = "testing_issues"

In [63]:
create_issues_for_dataset(
	dataset_id=dataset_id,
	dataset_name=dataset_name,
	token=token,
	owner=owner,
	repo=repo
)

Extracting the list of created datasets in the repo...
Retrieving issues from GitHub...
Extracting dataset IDs from issues...
Dataset ID WB_TEST_4 already exists. Skipping issue creation.


In [68]:
dataset_id = "WB_TEST_5"
dataset_name = "WB TEST 5 using API"
token = load_token("../../gihub.token")
owner = "WB-DECIS"
repo = "testing_issues"

In [69]:
create_issues_for_dataset(
	dataset_id=dataset_id,
	dataset_name=dataset_name,
	token=token,
	owner=owner,
	repo=repo
)

Extracting the list of created datasets in the repo...
Retrieving issues from GitHub...
Extracting dataset IDs from issues...
Creating issue for dataset ID WB_TEST_5...
[WB_TEST_5] - WB TEST 5 using API - ['Dataset']
Issue created successfully!
[WB_TEST_5] - Collection module - ['Collection', 'Epic']
Issue created successfully!
[WB_TEST_5] - Data curation - ['Modeling', 'Epic']
Issue created successfully!
[WB_TEST_5] - Metadata elements creation - ['Metadata elements', 'Epic']
Issue created successfully!
[WB_TEST_5] - Processing module - ['Processing', 'Epic']
Issue created successfully!
[WB_TEST_5] - Pipeline integration - ['Pip. integration', 'Epic']
Issue created successfully!
[WB_TEST_5] - Pipeline to prod - ['Pip. to prod', 'Epic']
Issue created successfully!
[WB_TEST_5] - Referential metadata review - ['Metadata review', 'Epic']
Issue created successfully!
[WB_TEST_5] - Data and metadata approval - ['Meta-data approval', 'Epic']
Issue created successfully!
[WB_TEST_5] - Publicati

True